In [20]:
import glob
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler

# 폴더 경로 지정
folder_path = r'C:\Users\pc\Desktop\CNN\CIC-AndMal2017'

In [21]:
# 랜섬웨어 클래스별 파일에 있는 샘플 개수를 확인하기 위한 딕셔너리
samples_per_class = {}

# Ransomware 폴더 내의 10개 폴더 경로 리스트 받아오기
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')

# 모든 파일 로드 및 샘플 개수 확인
for folder in ransomware_folder_list:
    ransomware_class = folder.split('\\')[-1]  # 랜섬웨어 클래스명 추출

    file_paths = glob.glob(folder + '\\*.csv')
    total_samples = 0  # 클래스별 전체 샘플 개수 초기화
    for file_path in file_paths:
        data = pd.read_csv(file_path)
        total_samples += data.shape[0]  # 데이터프레임의 행 수 / 샘플 개수 더하기

    samples_per_class[ransomware_class] = total_samples

# 클래스별 샘플 개수 출력
for ransomware_class, num_samples in samples_per_class.items():
    print(f"{ransomware_class} 클래스의 샘플 개수: {num_samples}")

Charger 클래스의 샘플 개수: 39551
Jisut 클래스의 샘플 개수: 25672
Koler 클래스의 샘플 개수: 44555
Lockerpin 클래스의 샘플 개수: 25307
Pletor 클래스의 샘플 개수: 4715
PornDroid 클래스의 샘플 개수: 46082
RansomBO 클래스의 샘플 개수: 39859
Simplocker 클래스의 샘플 개수: 36340
SVpeng 클래스의 샘플 개수: 54161
WannaLocker 클래스의 샘플 개수: 32701


In [22]:
benign_folder_path = r'C:\Users\pc\Desktop\CNN\CIC-AndMal2017\Benign'

# 'Benign' 클래스의 파일 경로 리스트 받아오기
benign_file_paths = glob.glob(benign_folder_path + '/*.csv')

# 'Benign' 클래스의 데이터프레임 초기화
benign_data = pd.DataFrame()

# 'Benign' 클래스의 모든 파일 로드
for file_path in benign_file_paths:
    data = pd.read_csv(file_path)
    benign_data = pd.concat([benign_data, data], ignore_index=True)

# 'Benign' 클래스에서 200,000개 무작위 샘플 추출
num_samples_benign = 200000
if len(benign_data) > num_samples_benign:
    benign_subset = benign_data.sample(n=num_samples_benign, random_state=42)
else:
    benign_subset = benign_data.copy()

print("benign_subset의 샘플 개수:", len(benign_subset))

benign_subset의 샘플 개수: 200000


In [23]:
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')
all_ransomware_data = pd.DataFrame()

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    
    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        all_ransomware_data = pd.concat([all_ransomware_data, data], ignore_index=True)

# 전체 랜섬웨어 데이터에서 200,000개 샘플 무작위로 추출
if len(all_ransomware_data) > 200000:
    all_ransomware_subset = all_ransomware_data.sample(n=200000, random_state=42, replace=False)
else:
    all_ransomware_subset = all_ransomware_data.copy()

print("all_ransomware_subset의 샘플 개수:", len(all_ransomware_subset))

all_ransomware_subset의 샘플 개수: 200000


In [24]:
# 'Benign' 클래스와 랜섬웨어 클래스의 데이터프레임들을 합치기
sub_dataset = pd.concat([benign_subset, all_ransomware_subset], ignore_index=True)

print("최종 특성추출 데이터세트의 샘플 개수:", len(sub_dataset))

최종 특성추출 데이터세트의 샘플 개수: 400000


In [25]:
# 타깃 변수 열 이름 확인
target_variable = ' Label'  # 실제 타깃 변수 열 이름으로 수정

# X와 y로 데이터 분할
y = sub_dataset[target_variable]  # 타깃 변수
X = sub_dataset.drop([target_variable], axis=1)  # 타깃 변수 제외한 나머지 특성

# 불필요한 특성 제거 (예시: 'Flow ID', ' Timestamp', ' Source IP', ' Destination IP' 특성 제거)
unnecessary_features = ['Flow ID', ' Timestamp', ' Source IP', ' Destination IP']
X = X.drop(unnecessary_features, axis=1)

# 각 열(feature)에 Min-Max 스케일링 적용
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# chi2를 사용하여 특성 선택 (하위 36개의 특성 선택)
num_features_to_select = 36
selector = SelectKBest(score_func=chi2, k=num_features_to_select)
X_selected = selector.fit_transform(X_scaled, y)

# 선택된 특성들의 인덱스 추출
selected_feature_indices = selector.get_support(indices=True)

# 선택된 특성들의 컬럼명 추출
selected_feature_names = X.columns[selected_feature_indices]

# 최종 선택된 특성들의 데이터프레임 생성
X_final = X[selected_feature_names]

# 결과 출력
print("선택된 특성들의 컬럼명:")
print(selected_feature_names)
print("\n최종 선택된 특성들의 데이터프레임:")
print(X_final.head())

선택된 특성들의 컬럼명:
Index([' Source Port', ' Destination Port', ' Flow Duration',
       'Total Length of Fwd Packets', ' Fwd Packet Length Max',
       ' Bwd Packet Length Std', ' Flow Packets/s', ' Flow IAT Mean',
       ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total',
       ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min',
       'Bwd IAT Total', ' Bwd IAT Max', 'Fwd PSH Flags', 'Fwd Packets/s',
       'FIN Flag Count', ' SYN Flag Count', ' PSH Flag Count',
       ' ACK Flag Count', ' URG Flag Count', ' Subflow Fwd Bytes',
       'Init_Win_bytes_forward', ' Init_Win_bytes_backward', 'Active Mean',
       ' Active Std', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Std',
       ' Idle Max', ' Idle Min'],
      dtype='object')

최종 선택된 특성들의 데이터프레임:
    Source Port   Destination Port   Flow Duration  \
0         38807                 53           95317   
1         49679                 80          258421   
2         36975                443        1024

In [26]:
# 특성들의 가중치(Chi-square 통계량) 확인
chi2_scores = selector.scores_[selected_feature_indices]

# 특성들의 가중치를 기준으로 오름차순 정렬
sorted_indices = chi2_scores.argsort()
sorted_features = selected_feature_names[sorted_indices]
sorted_chi2_scores = chi2_scores[sorted_indices]

# 특성들의 가중치와 순위 출력
print("특성들의 순위와 가중치:")
for i, (feature, score) in enumerate(zip(sorted_features, sorted_chi2_scores), 1):
    print(f"{i}. {feature}: {score}")

특성들의 순위와 가중치:
1.  Fwd Packet Length Max: 68.51133868993816
2.  Idle Std: 71.12354356709263
3. Total Length of Fwd Packets: 73.42559166162557
4.  Subflow Fwd Bytes: 73.42559166162557
5.  Active Min: 83.28418721520666
6.  Bwd IAT Max: 88.8963958229467
7.  Bwd Packet Length Std: 114.4627489105487
8. Bwd IAT Total: 127.52163666858264
9. Active Mean: 177.51414388595342
10.  Flow IAT Std: 185.62858152753807
11.  Init_Win_bytes_backward: 187.01606362293234
12. Fwd Packets/s: 223.70671692479854
13.  Fwd IAT Min: 224.95806560317286
14.  Flow Packets/s: 231.97393796382588
15.  Fwd IAT Mean: 238.4838937395082
16.  Fwd IAT Std: 251.5386000987008
17.  Flow IAT Min: 262.0408382976124
18.  Flow IAT Mean: 283.16758689898757
19.  Source Port: 298.84531536111183
20.  Idle Max: 303.9776781317766
21. Idle Mean: 304.2946892955119
22.  Idle Min: 312.0969996165415
23.  Active Max: 359.15956346435263
24.  Flow IAT Max: 379.90025003727374
25. Fwd PSH Flags: 393.5098629097659
26.  SYN Flag Count: 393.5098629097

In [27]:
benign_folder_path = r'C:\Users\pc\Desktop\CNN\CIC-AndMal2017\Benign'

# 'Benign' 클래스의 파일 경로 리스트 받아오기
benign_file_paths = glob.glob(benign_folder_path + '/*.csv')

# 'Benign' 클래스의 데이터프레임 초기화
benign_data = pd.DataFrame()

# 'Benign' 클래스의 모든 파일 로드
for file_path in benign_file_paths:
    data = pd.read_csv(file_path)
    benign_data = pd.concat([benign_data, data], ignore_index=True)

# 'Benign' 클래스에서 200,000개 무작위 샘플 추출
num_samples_benign = 200000
if len(benign_data) > num_samples_benign:
    benign_subset2 = benign_data.sample(n=num_samples_benign, random_state=42)
else:
    benign_subset2 = benign_data.copy()

print("benign_subset의 샘플 개수:", len(benign_subset2))

benign_subset의 샘플 개수: 200000


In [28]:
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')
all_ransomware_data = pd.DataFrame()

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    
    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        all_ransomware_data = pd.concat([all_ransomware_data, data], ignore_index=True)

# 전체 랜섬웨어 데이터에서 200,000개 샘플 무작위로 추출
if len(all_ransomware_data) > 200000:
    all_ransomware_subset2 = all_ransomware_data.sample(n=200000, random_state=42, replace=False)
else:
    all_ransomware_subset2 = all_ransomware_data.copy()

print("all_ransomware_subset의 샘플 개수:", len(all_ransomware_subset2))

all_ransomware_subset의 샘플 개수: 200000


In [29]:
from sklearn.model_selection import train_test_split

# 학습 및 임시 데이터 분할 (80% 학습, 20% 임시)
train_benign, temp_benign = train_test_split(benign_subset2, test_size=0.20, random_state=42)
train_ransomware, temp_ransomware = train_test_split(all_ransomware_subset2, test_size=0.20, random_state=42)

# 임시 데이터를 검증 및 테스트 데이터로 분할 (각각 50%, 합쳐서 20%를 10% 검증, 10% 테스트로 분할)
val_benign, test_benign = train_test_split(temp_benign, test_size=0.50, random_state=42)
val_ransomware, test_ransomware = train_test_split(temp_ransomware, test_size=0.50, random_state=42)

# 각 분할된 데이터 세트를 병합하여 최종 학습, 검증, 테스트 데이터 세트 생성
train_data = pd.concat([train_benign, train_ransomware], ignore_index=True)
val_data = pd.concat([val_benign, val_ransomware], ignore_index=True)
test_data = pd.concat([test_benign, test_ransomware], ignore_index=True)

print("Train samples:", len(train_data))
print("Validation samples:", len(val_data))
print("Test samples:", len(test_data))

Train samples: 320000
Validation samples: 40000
Test samples: 40000


In [30]:
# 선택된 36가지 특성에 해당하는 열만 추출
selected_feature_columns = X_final.columns

# 학습, 검증, 테스트 데이터 추출
X_train = train_data[selected_feature_columns]
X_val = val_data[selected_feature_columns]
X_test = test_data[selected_feature_columns]

# 결과 출력
print("학습 데이터:")
print(X_train.head())
print("\n검증 데이터:")
print(X_val.head())
print("\n테스트 데이터:")
print(X_test.head())

학습 데이터:
    Source Port   Destination Port   Flow Duration  \
0         37241                 80          128557   
1         47209                443         9164305   
2         58552                443            1044   
3         54797                 80        19693155   
4         58626                443        66444433   

   Total Length of Fwd Packets   Fwd Packet Length Max  \
0                        371.0                   371.0   
1                       1679.0                  1316.0   
2                         31.0                    31.0   
3                          0.0                     0.0   
4                        207.0                    23.0   

    Bwd Packet Length Std   Flow Packets/s   Flow IAT Mean   Flow IAT Std  \
0              546.173355        54.450555    2.142617e+04   1.895170e+04   
1              598.083197         2.073261    5.091281e+05   1.947289e+06   
2                0.000000      1915.708812    1.044000e+03   0.000000e+00   
3         

In [31]:
# 학습, 검증, 테스트 데이터의 타겟 변수 추출
y_train = train_data[' Label']
y_val = val_data[' Label']
y_test = test_data[' Label']

In [32]:
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 시작 시간 기록
start_time = time.time()

# 2. Decision Tree 학습 및 평가
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

dt_val_predictions = dt_classifier.predict(X_val)
print("Decision Tree - Validation Accuracy:", accuracy_score(y_val, dt_val_predictions))
print("Decision Tree - Validation Report:\n", classification_report(y_val, dt_val_predictions))

dt_test_predictions = dt_classifier.predict(X_test)
print("Decision Tree - Test Accuracy:", accuracy_score(y_test, dt_test_predictions))
print("Decision Tree - Test Report:\n", classification_report(y_test, dt_test_predictions))

# 종료 시간 기록 및 소요 시간 계산
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Decision Tree training and evaluation took {elapsed_time:.2f} seconds")

Decision Tree - Validation Accuracy: 0.465775
Decision Tree - Validation Report:
                         precision    recall  f1-score   support

                BENIGN       0.68      0.67      0.68     20000
    RANSOMWARE_CHARGER       0.21      0.23      0.22      2230
      RANSOMWARE_JISUT       0.18      0.18      0.18      1465
      RANSOMWARE_KOLER       0.30      0.31      0.30      2574
  RANSOMWARE_LOCKERPIN       0.16      0.17      0.16      1419
     RANSOMWARE_PLETOR       0.49      0.46      0.47       279
  RANSOMWARE_PORNDROID       0.27      0.27      0.27      2706
   RANSOMWARE_RANSOMBO       0.24      0.24      0.24      2260
 RANSOMWARE_SIMPLOCKER       0.22      0.23      0.23      2054
     RANSOMWARE_SVPENG       0.37      0.36      0.36      3169
RANSOMWARE_WANNALOCKER       0.23      0.24      0.23      1844

              accuracy                           0.47     40000
             macro avg       0.30      0.30      0.30     40000
          weighted a

In [33]:
# 시작 시간 기록
start_time = time.time()

# 3. Random Forest 학습 및 평가
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

rf_val_predictions = rf_classifier.predict(X_val)
print("Random Forest - Validation Accuracy:", accuracy_score(y_val, rf_val_predictions))
print("Random Forest - Validation Report:\n", classification_report(y_val, rf_val_predictions))

rf_test_predictions = rf_classifier.predict(X_test)
print("Random Forest - Test Accuracy:", accuracy_score(y_test, rf_test_predictions))
print("Random Forest - Test Report:\n", classification_report(y_test, rf_test_predictions))

# 종료 시간 기록 및 소요 시간 계산
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Random Forest training and evaluation took {elapsed_time:.2f} seconds")

Random Forest - Validation Accuracy: 0.5405
Random Forest - Validation Report:
                         precision    recall  f1-score   support

                BENIGN       0.62      0.86      0.72     20000
    RANSOMWARE_CHARGER       0.30      0.18      0.23      2230
      RANSOMWARE_JISUT       0.24      0.12      0.16      1465
      RANSOMWARE_KOLER       0.40      0.27      0.32      2574
  RANSOMWARE_LOCKERPIN       0.24      0.12      0.16      1419
     RANSOMWARE_PLETOR       0.57      0.46      0.51       279
  RANSOMWARE_PORNDROID       0.36      0.23      0.28      2706
   RANSOMWARE_RANSOMBO       0.32      0.20      0.24      2260
 RANSOMWARE_SIMPLOCKER       0.31      0.19      0.24      2054
     RANSOMWARE_SVPENG       0.49      0.33      0.39      3169
RANSOMWARE_WANNALOCKER       0.31      0.18      0.23      1844

              accuracy                           0.54     40000
             macro avg       0.38      0.29      0.32     40000
          weighted avg

In [34]:
from sklearn.neighbors import KNeighborsClassifier

# 시작 시간 기록
start_time = time.time()

# 4. K-Nearest Neighbors 학습 및 평가
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train, y_train)

knn_val_predictions = knn_classifier.predict(X_val)
print("KNN - Validation Accuracy:", accuracy_score(y_val, knn_val_predictions))
print("KNN - Validation Report:\n", classification_report(y_val, knn_val_predictions))

knn_test_predictions = knn_classifier.predict(X_test)
print("KNN - Test Accuracy:", accuracy_score(y_test, knn_test_predictions))
print("KNN - Test Report:\n", classification_report(y_test, knn_test_predictions))

# 종료 시간 기록 및 소요 시간 계산
end_time = time.time()
elapsed_time = end_time - start_time

print(f"KNN training and evaluation took {elapsed_time:.2f} seconds")

KNN - Validation Accuracy: 0.49125
KNN - Validation Report:
                         precision    recall  f1-score   support

                BENIGN       0.56      0.86      0.68     20000
    RANSOMWARE_CHARGER       0.21      0.13      0.16      2230
      RANSOMWARE_JISUT       0.16      0.06      0.09      1465
      RANSOMWARE_KOLER       0.36      0.19      0.25      2574
  RANSOMWARE_LOCKERPIN       0.14      0.05      0.07      1419
     RANSOMWARE_PLETOR       0.51      0.27      0.35       279
  RANSOMWARE_PORNDROID       0.29      0.14      0.19      2706
   RANSOMWARE_RANSOMBO       0.21      0.09      0.13      2260
 RANSOMWARE_SIMPLOCKER       0.22      0.09      0.13      2054
     RANSOMWARE_SVPENG       0.35      0.16      0.22      3169
RANSOMWARE_WANNALOCKER       0.23      0.07      0.11      1844

              accuracy                           0.49     40000
             macro avg       0.29      0.19      0.22     40000
          weighted avg       0.41      0.